In [1]:
import jax
jax.config.update("jax_platform_name", "cpu")
jax.config.update("jax_enable_x64", True)
import numpy as np
np.set_printoptions(precision=3, linewidth=150)
import spax
from datetime import datetime
from sklearn.decomposition import PCA

# Principle Component Analysis (PCA)
Let X be $n \times m$ matrix, where $n$ represents dimensionality of the data and $m$ number of samples.
To prepare data for the PCA, it should be centered `X -= X.mean(axis = 1)`, and possibly whitened `X /= X.std(axis = 1)`. Whitening is done to scale dimensions to unit variance. This is specifically useful if data dimensions are in different units or represent different observables. PCA then consists of finding a rotation matrix which makes covariance of the data diagonal. Specifically, PCA of order $N$ does this rotation for $N$ largest eigenvalues of the covariance matrix.

## Singular Value Decomposition (SVD)
Every non-quadratic matrix can be decomposed as $$X = U \, S \, V^T \, ,$$ where $U$ is $n \times m$ matrix with orthonormal columns ($U^T \, U = I$), $V$ is $m \times m$ orthonormal matrix ($V^T \, V = I$) and $S$ a diagonal $m\times m$ matrix of singular values. From such decomposition one can write
\begin{align}
X \, X^T &= U \, S^2 \, U^T  \, , \\
X^T X &= V \, S^2 \, V^T \, .
\end{align}
Both $X \, X^T$ and $X^T X$ have the same eigenvalues, with larger of the two having the rest equal to $0$.

## Case #1: $n \le m$
Covariance of the data can be written as $$ C = \frac{1}{m-1} X \, X^T = U \, \frac{S^2}{m-1} \, U^T \, .$$ Therefore, by solving an eigenvalue problem for $C$, we can find $U$. By picking $N$ eigenvectors in the directions of the largest eigenvalues, we construct $\widetilde{U}$ used for PCA.

For some matrix $X_0$ of size $n \times m_0$, PCA is simply $$ Y_0 = \widetilde{U}^T X_0$$.

In [2]:
N_dim, N_samples = (8, 10**5)
pca = spax.PCA_m(5, devices = jax.devices("gpu"))
data = np.random.normal(0, 1, size = (N_dim, N_samples)) * np.sqrt(np.arange(1, N_dim + 1))[:, np.newaxis]
tic = datetime.now()
pca.fit(data, batch_size = N_dim // 2) # N_dim % (N_devices * batch_size) == 0
print("DURATION:", datetime.now() - tic)
sampled_data = pca.sample(N_samples)
print(np.std(pca.transform(sampled_data), axis = 1)**2) # should be [4, 5, 6, 7, 8]
print(pca.λ ** 2) # should be the same
print(np.round(pca.U.T, 1)) # should be a +-unit matrix on last 5 dimensions

DURATION: 0:00:03.406254
[4.006 5.024 6.028 7.02  8.026]
[4.024 5.005 6.012 7.017 8.041]
[[ 0. -0. -0.  1.  0. -0.  0.  0.]
 [-0. -0.  0. -0.  1. -0. -0. -0.]
 [-0. -0. -0.  0.  0.  1. -0.  0.]
 [ 0. -0.  0. -0.  0.  0.  1. -0.]
 [ 0. -0.  0. -0.  0. -0.  0.  1.]]


In [3]:
#testing the result with scikit-learn
pca_sk = PCA(n_components = 5)
tic = datetime.now()
pca_sk.fit(data.T)
print("DURATION:", datetime.now() - tic)
print(np.std(pca_sk.transform(sampled_data.T), axis = 0)[::-1]**2)
print(pca_sk.singular_values_[::-1]**2 / (N_samples - 1))
print(np.round(pca_sk.components_[::-1], 1))

DURATION: 0:00:00.168808
[4.007 5.023 6.028 7.019 8.026]
[4.024 5.006 6.012 7.016 8.041]
[[-0.  0.  0. -1.  0. -0.  0. -0.]
 [-0. -0.  0.  0.  1. -0. -0. -0.]
 [ 0.  0.  0.  0. -0. -1.  0. -0.]
 [-0.  0. -0. -0. -0. -0. -1.  0.]
 [-0.  0. -0. -0.  0. -0.  0.  1.]]


## Case #2 $n \ge m$
In this case, it is better to write: $$D = \frac{1}{n} X^T \, X = V \, \frac{S^2}{n} \, V^T \, .$$
Solving eigenvector problem for $D$ gives us $V$ and $S$. Then, rotation matrix can be computed as $$ U = X \, V \, S^{-1} \, .$$ The rest is the same as in previous case.

In [4]:
N_dim, N_samples = (10**5, 8)
pca = spax.PCA_m(5, devices = jax.devices("gpu"))
data = np.random.normal(0, 1, size = (N_dim, N_samples)) * np.sqrt(np.arange(1, N_dim + 1))[:, np.newaxis]
tic = datetime.now()
pca.fit(data, batch_size = N_dim // 2, centering_data = "CPU") # N_dim % (N_devices * batch_size) == 0
print("DURATION:", datetime.now() - tic)
sampled_data = pca.sample(N_samples)
print(np.std(pca.transform(sampled_data), axis = 1)**2)
print(pca.λ ** 2)
print(pca.U.T)

DURATION: 0:00:01.388217
[7.105e+08 7.095e+08 3.575e+08 8.420e+08 6.250e+08]
[7.107e+08 7.124e+08 7.194e+08 7.215e+08 7.256e+08]
[[-5.837e-06 -2.191e-05  1.284e-05 ...  2.404e-03  3.513e-03  2.703e-03]
 [-2.905e-05 -6.952e-06  1.859e-05 ...  3.931e-03 -7.002e-04 -1.381e-03]
 [-2.901e-05  3.567e-05 -4.768e-07 ...  4.191e-03  9.461e-04  3.027e-03]
 [ 4.215e-06  2.570e-05 -3.951e-05 ... -1.565e-03  6.404e-03  1.125e-03]
 [-2.866e-06  6.922e-06 -1.386e-06 ...  1.091e-03 -3.548e-04  1.937e-03]]


In [5]:
pca_sk = PCA(n_components = 5)
tic = datetime.now()
pca_sk.fit(data.T)
print("DURATION:", datetime.now() - tic)
print(np.std(pca_sk.transform(sampled_data.T), axis = 0)[::-1]**2)
print(pca_sk.singular_values_[::-1]**2 / (N_samples - 1))
print(pca_sk.components_[::-1])

DURATION: 0:00:00.118943
[7.106e+08 7.095e+08 3.575e+08 8.420e+08 6.250e+08]
[7.107e+08 7.124e+08 7.194e+08 7.215e+08 7.256e+08]
[[ 5.839e-06  2.191e-05 -1.285e-05 ... -2.404e-03 -3.513e-03 -2.703e-03]
 [ 2.905e-05  6.950e-06 -1.859e-05 ... -3.931e-03  7.004e-04  1.382e-03]
 [-2.901e-05  3.567e-05 -4.777e-07 ...  4.191e-03  9.462e-04  3.027e-03]
 [-4.215e-06 -2.569e-05  3.951e-05 ...  1.565e-03 -6.404e-03 -1.124e-03]
 [-2.866e-06  6.922e-06 -1.387e-06 ...  1.091e-03 -3.547e-04  1.937e-03]]


## Test save + load

In [6]:
pca.save("test.hdf5")
pca_new = spax.PCA_m(5, devices = jax.devices("gpu"))
pca_new.load("test.hdf5")

x0 = np.arange(100000)[:, np.newaxis]
print(pca.transform(x0).flatten())
print(pca_new.transform(x0).flatten())
print(pca_sk.transform(x0.T)[:, ::-1].flatten())

[-65054.42  -71685.87   16007.564  65883.86  -91552.93 ]
[-65054.42  -71685.87   16007.564  65883.86  -91552.93 ]
[ 65060.055  71681.08   16010.391 -65885.033 -91551.482]
